# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import pickle
import sys
import math
import os
import random
import itertools

# Splitting dataset into train and test set
The dataset was split into half to form a train and test split. Since it is essential to have half of each user’s rated movies in training and the other half as testing and we have a total of 138493 users, it was computationally infeasible to perform the split randomly for each user. Hence, the even rows in the dataset were included in the training set and the odd rows were included in the test set. Although not random, it ensures that half of each user’s rated movies are in the training set and the other half in the testing set since all rated movies for a particular user in the dataset are contiguous.

In [311]:
def createTestSet(df, splitFraction, seed):
    return df.drop(df.sample(frac=splitFraction, random_state=seed).index)

def splitDatasets(splitFraction):
    df = pd.read_csv('ml-20m/ratings.csv')
    print("Total size of dataset: ", len(df))

    trainDf = df.iloc[[i for i in range(0,len(df),2)]]
    testDf = df.iloc[[i for i in range(1,len(df),2)]]

    print("Training set size: ", len(trainDf))
    print("Test set size: ", len(testDf))

    return trainDf, testDf

('Total size of dataset: ', 20000263)
('Training set size: ', 10000132)
('Test set size: ', 10000131)


In [ ]:
trainDf, testDf = splitDatasets(splitFraction=0.5)
trainDf.to_csv('processed/train.csv', index=False)
testDf.to_csv('processed/test.csv', index=False

# Building Movie Dictionary from dataset
Since the movie ids were not contiguous (unlike user ids), they had to be mapped to contiguous integers which can be used to index the columns of the rating matrix R and the rows of its factor W. This was done by finding the unique movie ids in the entire dataset and then mapping them to contiguous integers using a dictionary. This was stored as a pickle file so that the dictionary just needs to be computed once in the beginning.

In [4]:
def buildMovieDict(df):
    distinctMovies = df['movieId'].unique()
    numMovies = len(distinctMovies)
    movieDict = dict([(distinctMovies[i], i + 1) for i in range(0, numMovies)])
    
    with open('intermediate/movieDict.pkl', 'wb') as handle:
        pickle.dump(movieDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return movieDict

In [281]:
df = pd.read_csv('ml-20m/ratings.csv')
movieDict = buildMovieDict(df)

# Building ratings matrix R from dataframe
The next step was to implement a function to create the ratings matrix R from a dataset (train set or test set in this case). This was implemented as a sparse matrix in scipy due to the large dimensions of R and the sparsity of the observed ratings.

In [3]:
def buildRatingsMatrix(df, movieDict):
    R = sps.coo_matrix((df['rating'], (df['userId'], df['movieId'].apply(lambda cell: movieDict[cell]))))
    print("Rating Matrix shape: ", R.shape)
    return R

# Saving factors of R
The next step was to implement a function to save the factors V and W in a pickle file after the training is completed.

In [5]:
def saveFactors(V, W):
    with open('factors/V.pkl', 'wb') as handle:
        pickle.dump(V, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('factors/W.pkl', 'wb') as handle:
        pickle.dump(W, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Initializing / Loading factors of R
If the factors V and W are already stored in a pickle file, they are loaded from the file instead of randomly initializing V and W. This ensures that training can be resumed later and it does not need to start from scratch every time.

In [6]:
def initFactors(numUsers, numMovies, rank):
    V = np.random.randn(numUsers + 1, rank)
    W = np.random.randn(numMovies + 1, rank)
    
    if(os.path.exists('factors/V.pkl')):
        print("V exists")
        with open('factors/V.pkl', 'rb') as handle:
            V = pickle.load(handle)
    if(os.path.exists('factors/W.pkl')):
        print("W exists")
        with open('factors/W.pkl', 'rb') as handle:
            W = pickle.load(handle)
            
    return V, W

# Computing Cost

In [7]:
def findCost(R, V, W, decay):
    cost = np.sqrt(len(R.row)) * (findRMSE(R, V, W))
    
    squared_V_norm = np.linalg.norm(V)**2
    squared_W_norm = np.linalg.norm(W)**2
    
    cost = cost + decay * (squared_V_norm + squared_W_norm)    
    return cost

# Computing RMSE

In [9]:
def findRMSE(R, V, W):
    mse = 0
    size = len(R.row)
    
    for row, col, r in itertools.izip(R.row, R.col, R.data):
        if col >= len(W):
            vw_T = 2.5
        else:
            v = V[row]
            w = W[col]
            vw_T = v.dot(w)
        r_minus_vwT = (r - vw_T)        
        mse = mse + r_minus_vwT**2
    
    rmse = np.sqrt(mse/size)
    return rmse

# Computing MRR

In [108]:
def findMRR(R_test, testDf, V, W):
    filteredDf = testDf[testDf['rating'] >= 3]
    filteredTest = buildRatingsMatrix(filteredDf, movieDict).tolil()
    
    numUsers = R_test.shape[0] - 1
    mrr = 0
    count = 0
    
    for user in range(1, numUsers + 1):
        if(user % 20000 == 0):
            print("User id: ", user)
            print("MRR: ", mrr/float(count))

        v = V[user]
        r_predicted = np.matmul(v, W.T).flatten()
        
        # appends two minimum ratings for the two movies present in test set but not in train set
        # since those movies should have minimum chance of being recommended
        r_predicted = np.append(r_predicted, r_predicted.min()-1)
        r_predicted = np.append(r_predicted, r_predicted.min()-1)
        
        sortedIndices = np.argsort(-r_predicted)
        indices_of_sortedIndices = filteredTest.rows[user]
        
        ranks = sortedIndices[indices_of_sortedIndices] + 1
        inv_ranks = 1.0/ranks
        
        user_mrr = 0
        
        if len(inv_ranks.flatten()) > 0:
            user_mrr = np.sum(inv_ranks)/float(len(inv_ranks.flatten()))
            count = count + 1
        
        mrr = mrr + user_mrr
        
    mrr = mrr/float(count)
    print("MRR: ", mrr)
    return mrr      

# Training
The main function for training using stochastic gradient descent with regularization. An entire pass of the training set is done in each epoch.

In [109]:
def train(R, learning_rate, decay, epochs, rank):
    numUsers = R.shape[0] - 1
    numMovies = R.shape[1] - 1
    
    # initialise/load V and W
    V, W = initFactors(numUsers, numMovies, rank)
        
    try:
        j = 0
        
        # train for multiple epochs
        for i in range(0, epochs):
            # iterate through all rated movies in R in each epoch
            for row, col, r in itertools.izip(R.row, R.col, R.data):
                if(j % 10000000 == 0):
                    print("Iteration: ", (j + 1))

                j = j + 1
                
                # find predicted rating for a single user-movie pair
                v = V[row]
                w = W[col]
                vw_T = v.dot(w)
                
                # find difference between true rating and predicted rating of the user-movie pair
                r_minus_vwT = r - vw_T

                # update v and w with regularization using SGD for the user and movie
                v_new = v + learning_rate * (2 * r_minus_vwT * w - 2 * decay * v)
                w_new = w + learning_rate * (2 * r_minus_vwT * v - 2 * decay * w)

                V[row] = v_new
                W[col] = w_new

        # find cost
        cost = findCost(R, V, W, decay)
        print("Cost: ", cost)
        
        # save V and W
        saveFactors(V, W)
            
    except KeyboardInterrupt:
        # find cost
        cost = findCost(R, V, W, decay)
        print("Cost: ", cost)
        
        # save and return V and W
        saveFactors(V, W)
        return V, W

    return V, W

# Main Program

In [12]:
trainDf = pd.read_csv('processed/train.csv')
with open('intermediate/movieDict.pkl', 'rb') as handle:
    movieDict = pickle.load(handle)
R = buildRatingsMatrix(trainDf, movieDict)

Index([u'userId', u'movieId', u'rating', u'timestamp'], dtype='object')
10000132
('Rating Matrix shape: ', (138494, 26743))


In [66]:
%%time
V, W = train(R, 0.02, 0.05, 1, 8)

V exists
W exists
('Iteration: ', 1)
('Iteration: ', 10000001)
('Cost: ', 36569.29635013777)
Wall time: 2min 20s


In [15]:
testDf = pd.read_csv('processed/test.csv')
R_test = buildRatingsMatrix(testDf, movieDict)

Index([u'userId', u'movieId', u'rating', u'timestamp'], dtype='object')
10000131


In [67]:
%%time
print("Test rmse: ", findRMSE(R_test, V, W))

('Test rmse: ', 0.8991843561757922)
Wall time: 21.4 s


In [107]:
%%time
mrr = findMRR(R_test, testDf, V, W)

('Rating Matrix shape: ', (138494, 26745))
('User id: ', 20000)
('MRR: ', 0.0014721004876985387)
('User id: ', 40000)
('MRR: ', 0.001461298352154925)
('User id: ', 60000)
('MRR: ', 0.0014485085738971488)
('User id: ', 80000)
('MRR: ', 0.001434079726996797)
('User id: ', 100000)
('MRR: ', 0.0014388180353729094)
('User id: ', 120000)
('MRR: ', 0.001439528230753022)
('MRR: ', 0.0014466085492235278)
Wall time: 6min 45s


# Results

**The format for presenting the results is as shown below:**
<br>
**[params] - cost, rmse, mrr**
<br>
<br>
**The results for different parameters are as shown below:**
<br>
alpha=0.02, lambda=0.05, epochs=4, rank=8 - 35716.07382967673, 0.937433774076947, 0.0016055583979725132
<br>
alpha=0.02, lambda=0.05, epochs=5, rank=8 - 36569.29635013777, 0.8991843561757922, 0.0014466085492235278
<br>
<br>
alpha=0.02, lambda=0.05, epochs=3, rank=6 - 57956.23828272209, 2.3831375938103045, 0.00031533441247997315
<br>
alpha=0.02, lambda=0.05, epochs=3, rank=8 - 64802.70846971691, 2.1481592303174124, 0.00024105607546223403
<br>
alpha=0.02, lambda=0.05, epochs=3, rank=10 - 73383.93550978332, 2.2577263788179387, 0.00021896018357628571
<br>
alpha=0.02, lambda=0.05, epochs=3, rank=12 - 78496.42261932473, 1.722586040283862, 0.00019679348599852103
<br>
alpha=0.02, lambda=0.05, epochs=3, rank=14 - 86365.03847632423, 1.769643054603345, 0.0001635216989103651
<br>
<br>
alpha=0.02, lambda=0.03, epochs=3, rank=12 - 51898.65484773593, 1.80567572792333, 0.0002782428808433028
<br>
alpha=0.02, lambda=0.06, epochs=3, rank=12 - 91243.59263266146, 1.6540362682154743, 0.00020567054655245604
<br>
alpha=0.02, lambda=0.07, epochs=3, rank=12 - 101825.6501196088, 1.4577480021398017, 0.0002127411669789362
<br>
alpha=0.02, lambda=0.08, epochs=3, rank=12 - 114734.08176509904, 1.8308525526425596, 0.00023944179999416633
<br>
<br>
alpha=0.01, lambda=0.07, epochs=3, rank=12 - 116822.95796665098, 1.9511333472712422, 0.00020466758972919683
<br>
alpha=0.015, lambda=0.07, epochs=3, rank=12 - 111242.52258992748, 2.3667979083432162, 0.00018848672450557549
<br>
alpha=0.025, lambda=0.07, epochs=3, rank=12 - 97016.95650144889, 1.4949732339925546, 0.00018760914758979196
<br>
alpha=0.03, lambda=0.07, epochs=3, rank=12 - 95808.92566118314, 2.090566906821871, 0.00020254878287447923
<br>
<br>
alpha=0.02, lambda=0.07, epochs=8, rank=12 - 81004.78937769009, 1.0790033312432232, 0.0002654475673025698